In [21]:
from threading import Thread,Semaphore
import numpy as np
import random
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from collections import deque

In [22]:
Lambda = 1
global SZ,CP,RM
scale = 1000000
SZ=[]
CP=[]
RM=[]
N=10000
minibatch = 32
k=10000
C=5
state=[]
#Lambda = 10
#mu = 1.5
alpha = 0.3
gamma = 0.95
#t = 0   #system clock
t1=0
t2=0
t_lambda = 0 #poisson random variable used to generate next service time
t_mu = 0  #poisson random variable used to generate next departure time
t_a=0
wait=0
R=100

transmission_time=0
l="0"
P_transmission=500
P_idle=100

episodes = 10
SHOW_EVERY = 100
STATS_EVERY=100

Q_dict=dict()
L=[]

action_list=()

train_data=[]
train_label=[]

In [23]:
def reward_generator( T, cpu, bw, m):
    cpuU=cpu*0.5
    mU=m*0.5
    bwU=bw*0.3

    cpusla=680
    bwsla=301
    msla=1000
    Tsla1=1.5
    Tsla2=3

    Ct1=0
    Ct2=0
    Cp=0
    Cbw=0
    Cm=0
    rt=0
    ru=0
    

    
    #if (G<Gsla):
        #Cg=-5*(Gsla/G)
    if (cpuU<cpusla):
        Cp=-4* (cpusla/cpuU)
    if (mU<msla):
        Cm=-4* (msla/mU)
    if (bwU<bwsla):
        Cbw=-4* (bwsla/bwU)
    if (T>Tsla1 and T<Tsla2):
        Ct1 = -5 * (T/Tsla1)
        rt=(Tsla1/T) + Ct1
    elif (T>=Tsla2):
        Ct2 = -6 * (T/Tsla1)
        rt=(Tsla1/T) + Ct2
    else:
        rt=(Tsla1/T)
        

    #if (N>Nsla):
        #Cn=-5*(N/Nsla)
    #if (cost>Cost_sla):
        #Cost_n=-5*(cost/Cost_sla)
    ru=(cpuU/cpusla)+Cp+(bwU/bwsla)+Cbw+(mU/msla)+Cm
    Reward = rt + ru   #Reward Function
    return Reward

In [24]:
class Packet(object):
    def __init__(self, mobileID):
                                                        
        self.mobileID=mobileID   
        self.size=random.randint(200,400) #seed 1 op 2, seed 2 op 
        SZ.append(round(self.size,2))
        self.cpucycle=random.uniform(100,1900)
        self.memory_need=random.uniform(1000,2000)
        self.ram=random.randint(1, 32) #seed 1 op 3, seed 2 op 1, seed 3 op 4
        RM.append(round(self.ram,2))
        
        
class Mobile():
    
    def __init__(self,id):
        
        self.id = id
        
    def packet_generator(self):
        
        p = Packet(self.id)
        
        return p

In [25]:
class Q_Learning():
    
    def __init__(self,gamma,alpha):
        
        self.d = dict()
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = 1
        self.epsilon_min = 0.1
        
    def calculate_Q_value(self,a,prevState,newState,reward):
        
        self.d[(prevState,a)] = self.d.get((prevState,a),0) + self.alpha*(reward + self.gamma * max(self.d.get((newState,0),0),self.d.get((newState,1),0),self.d.get((newState,2),0)) - self.d.get((prevState,action),0))
        
    def get_action(self,state):
        
        return np.argmax([self.d.get((state,0),0),self.d.get((state,1),0),self.d.get((state,2),0)])
        
        

In [26]:
class DQNAgent:
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.memory=deque(maxlen=20000)
        self.gamma=0.99
        self.epsilon=1.0
        self.epsilon_decay=0.995
        self.epsilon_min=0.01
        self.learning_rate=0.000025
        self.model=self.build_model()
        self.target_model=self.build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.target_update_counter=0
        
    def build_model(self):
        model=Sequential()
        model.add(Dense(8,input_dim=self.state_size,activation='relu'))
        model.add(Dense(8,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self,state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state=np.expand_dims(state, axis=0)
        
        act_values=self.model.predict(state)
        return np.argmax(act_values[0])
        
        
    def replay(self,batch_size):

        minibatch=random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            target=reward
            state=np.expand_dims(state, axis=0)
            next_state=np.expand_dims(next_state, axis=0)
            if not done:
                target=reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            target_f=self.model.predict(state)
            target_f[0][action]=target
            self.model.fit(state,target_f,epochs=1,verbose=0)
            
        if self.epsilon>self.epsilon_min:
            self.epsilon*=self.epsilon_decay
    
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def load(self,name):
        self.model.load_weights(name)
        
    def save(self,name):
        self.model.save_weights(name)


In [27]:
class Fog:
    
    def __init__(self,num_server,mu,k,ID):
        
        self.num_server = num_server
        self.mu = mu
        self.lock = Semaphore(self.num_server)
        self.Serverqueue = 0
        self.waitTime = []
        self.responseTime = []
        self.reward_list = []
        self.cpuspeed=0
        self.cpucycle=0
        self.memory=0
        self.R=0
        self.ID = ID
        self.neighbour_list = []
        
        self.lost = 0
        self.transmission_time = 0
        
    def run(self, packet, State):
        self.R=random.uniform(900,1100)
        self.transmission_time = (packet.size/self.R)
        
        time.sleep(((-np.log(random.random())/Lambda)+self.transmission_time)/scale)

        t = Thread(target=self.worker,args=(time.time(),packet,State))
        
        self.Serverqueue+=1
        t.start()
        
        
        
    def getState(self):
        
        return (self.Serverqueue, self.cpuspeed, self.R, self.memory)
    
    def set_neighbour(self,n1,n2):
        self.neighbour_list.append(n1)
        self.neighbour_list.append(n2)
    
    
    def worker(self,arrival,packet,State):
        
        r = 0
        
        new_cpuspeed = 0
        new_mem = 0
        new_queuelength = 0
        
        if self.lock.acquire():
            
            self.cpuspeed=random.uniform(1000,1500)
            new_cpuspeed = self.cpuspeed
            
            
            self.memory=random.uniform(3000,4000)
            self.memory=self.memory-packet.memory_need 
            new_mem = self.memory
            
            
            self.waitTime.append((time.time()-arrival)*scale)
            
            t_mu=packet.cpucycle/self.cpuspeed
            
            time.sleep(t_mu/scale)
            
            response_time = time.time() - arrival
            
            self.responseTime.append(response_time*scale)
            
            r=reward_generator(response_time,self.cpuspeed,self.R,self.memory)
            self.reward_list.append(r)
            
            self.Serverqueue-=1
            new_queuelength = self.Serverqueue
            
            
 
        self.lock.release()
        new_state = (new_queuelength,new_cpuspeed,self.R,new_mem) + self.neighbour_list[0].getState() + self.neighbour_list[1].getState()
        deep_Q.remember(State,self.ID,r,new_state,False)
        # Qlearning.calculate_Q_value(self.ID,State,new_state,r)

In [28]:
c1 = Fog(3,1.25,10,0)
c2 = Fog(3,1.25,10,1)
c3 = Fog(3,1.25,10,2)

c1.set_neighbour(c2,c3)
c2.set_neighbour(c1,c3)
c3.set_neighbour(c1,c2)

m1 = Mobile(1)


In [20]:
startSim = time.time()
# Qlearning = Q_Learning(0.95,0.1)
deep_Q = DQNAgent(12,3)

C = 0
# state 1
for i in range(1000000):
    print("Iteration:",i)
    p1 = m1.packet_generator()
#     p2 = m2.packet_generator()
#     p3 = m3.packet_generator()
    
    State = c1.getState() + c2.getState() + c3.getState() 
    
    if i<=20000:
        action = random.randint(0,2)
    else:
        action = deep_Q.act(State)
        # action = np.random.choice([action,random.randint(0,2)], p=[1 - Qlearning.epsilon , Qlearning.epsilon])
    
    if action == 0:
        c1.run(p1,State)
    elif action == 1:
        c2.run(p1,State)
    elif action == 2:
        c2.run(p1,State)

    if len(deep_Q.memory) >= minibatch :
      deep_Q.replay(minibatch)
    
    if C==32:
      deep_Q.update_target_network()
      C=0

    C+=1

    # if Qlearning.epsilon>Qlearning.epsilon_min:
    #     Qlearning.epsilon = Qlearning.epsilon - (Qlearning.epsilon*0.01)
    # else:
    #     Qlearning.epsilon = 0.1
    
print("Simulation time:",time.time()-startSim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
Iteration: 15004
Iteration: 15005
Iteration: 15006
Iteration: 15007
Iteration: 15008
Iteration: 15009
Iteration: 15010
Iteration: 15011
Iteration: 15012
Iteration: 15013
Iteration: 15014
Iteration: 15015
Iteration: 15016
Iteration: 15017
Iteration: 15018
Iteration: 15019
Iteration: 15020
Iteration: 15021
Iteration: 15022
Iteration: 15023
Iteration: 15024
Iteration: 15025
Iteration: 15026
Iteration: 15027
Iteration: 15028
Iteration: 15029
Iteration: 15030
Iteration: 15031
Iteration: 15032
Iteration: 15033
Iteration: 15034
Iteration: 15035
Iteration: 15036
Iteration: 15037
Iteration: 15038
Iteration: 15039
Iteration: 15040
Iteration: 15041
Iteration: 15042
Iteration: 15043
Iteration: 15044
Iteration: 15045
Iteration: 15046
Iteration: 15047
Iteration: 15048
Iteration: 15049
Iteration: 15050
Iteration: 15051
Iteration: 15052
Iteration: 15053
Iteration: 15054
Iteration: 15055
Iteration: 15056
Iteration: 15057
Iteration: 15058
Iteration: 150

KeyboardInterrupt: ignored